# Language abstraction analysis notebook

## Corpus

This notebook collects analyses performed for the LAX cogsci 2022 paper, as well as some additional analyses.

Statistics were largely done outside of this notebook, in R (see lax_corpus_statistics.Rmd and mixed-effects-model.Rmd in this folder).

In [ ]:
subdomains = {
    'structures' :  ['bridge','city','house','castle'],
    'drawing' :  ['nuts-bolts','wheels','dials','furniture']
}

domains = list(subdomains.keys())

In [ ]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
# sys.path.append("../utils")
sys.path.append("../../../stimuli")

import numpy as np
import scipy.stats as stats
import scipy.spatial.distance as distance
import pandas as pd

from collections import Counter
from functools import reduce
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont, ImageColor

from io import BytesIO
import base64

import random
import  matplotlib
from matplotlib import pylab, mlab, pyplot
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('whitegrid')

from IPython.display import clear_output, Image, HTML

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import spacy
nlp = spacy.load("en_core_web_lg")
# sys.path.append("../../stimuli/towers/block_utils/")
# import blockworld_utils as utils

In [ ]:
# styling for paper_figures

sns.set_style('white', {'axes.linewidth': 0.5})
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True

LIGHT_BLUE = "#56B0CD"
LIGHT_ORANGE = "#FFCE78"
LIGHT_GREEN = "#95C793"
LIGHT_RED = "#CC867A"

BLUE = "#009BCD"
ORANGE = "#FFA300"
GREEN = "#688B67"
RED = "#CC5945"

DARK_BLUE   = "#0E4478"
DARK_ORANGE = "#A46400"
DARK_GREEN  = "#275C4A"
DARK_RED    =  "#9B3024"

domain_palettes_light = {
    
    domains[0]:{
        subdomains[domains[0]][0]: LIGHT_BLUE,
        subdomains[domains[0]][1]: LIGHT_ORANGE,
        subdomains[domains[0]][2]: LIGHT_GREEN, 
        subdomains[domains[0]][3]: LIGHT_RED   
    },
     domains[1]:{
        subdomains[domains[1]][0]: LIGHT_BLUE,
        subdomains[domains[1]][1]: LIGHT_ORANGE,
        subdomains[domains[1]][2]: LIGHT_GREEN, 
        subdomains[domains[1]][3]: LIGHT_RED 
    }
}

domain_palettes = {
    
    domains[0]:{
        subdomains[domains[0]][0]: BLUE,
        subdomains[domains[0]][1]: ORANGE,
        subdomains[domains[0]][2]: GREEN,
        subdomains[domains[0]][3]: RED
    },
     domains[1]:{
        subdomains[domains[1]][0]: BLUE,
        subdomains[domains[1]][1]: ORANGE,
        subdomains[domains[1]][2]: GREEN,
        subdomains[domains[1]][3]: RED
    }
}

domain_palettes_dark = {
    
    domains[0]:{
        subdomains[domains[0]][0]: DARK_BLUE,
        subdomains[domains[0]][1]: DARK_ORANGE,
        subdomains[domains[0]][2]: DARK_GREEN, 
        subdomains[domains[0]][3]: DARK_RED   
    },
     domains[1]:{
        subdomains[domains[1]][0]: DARK_BLUE,
        subdomains[domains[1]][1]: DARK_ORANGE,
        subdomains[domains[1]][2]: DARK_GREEN, 
        subdomains[domains[1]][3]: DARK_RED 
    }
}

N=256
gradients = []

for light, mid, dark in zip([LIGHT_BLUE,LIGHT_ORANGE,LIGHT_GREEN,LIGHT_RED],[BLUE,ORANGE,GREEN,RED],[DARK_BLUE,DARK_ORANGE,DARK_GREEN,DARK_RED]):
    light_rgb = list(ImageColor.getcolor(light, "RGB"))
    mid_rgb = list(ImageColor.getcolor(mid, "RGB"))
    dark_rgb = list(ImageColor.getcolor(dark, "RGB"))
    vals = np.ones((N, 4))
    vals[:, 0] = np.append(np.linspace(light_rgb[0]/255, mid_rgb[0]/255, int(N/2)),np.linspace(mid_rgb[0]/255, dark_rgb[0]/255, int(N/2))) # R
    vals[:, 1] = np.append(np.linspace(light_rgb[1]/255, mid_rgb[1]/255, int(N/2)),np.linspace(mid_rgb[1]/255, dark_rgb[1]/255, int(N/2))) # G
    vals[:, 2] = np.append(np.linspace(light_rgb[2]/255, mid_rgb[2]/255, int(N/2)),np.linspace(mid_rgb[2]/255, dark_rgb[2]/255, int(N/2))) # B
    newcmp = ListedColormap(vals)
    
    gradients.append(newcmp)

domain_gradients = {

    domains[0]:{
        subdomains[domains[0]][0]: gradients[0],
        subdomains[domains[0]][1]: gradients[1],
        subdomains[domains[0]][2]: gradients[2],
        subdomains[domains[0]][3]: gradients[3],
    },
     domains[1]:{
        subdomains[domains[1]][0]: gradients[0],
        subdomains[domains[1]][1]: gradients[1],
        subdomains[domains[1]][2]: gradients[2],
        subdomains[domains[1]][3]: gradients[3],
    }
}


In [ ]:
def add_numbers_and_space(responses):
    responses = [f"{id}: {response}" for (id, response) in enumerate(responses)]
    responses = '\n'.join(responses)
    return responses

def group_by_stim_url(df, config_name):
    df[config_name] = df[['stimURL','responses']].groupby(['stimURL'])['responses'].transform(lambda responses: add_numbers_and_space(responses))
    df[['stimURL', config_name]].drop_duplicates()
    return df[['stimURL', config_name]]

def group_by_stim_id(df, config_name):
    df[config_name] = df[['stimId','responses']].groupby(['stimId'])['responses'].transform(lambda responses: add_numbers_and_space(responses))
    df[['stimId', config_name]].drop_duplicates()
    return df[['stimId', config_name]]

def path_to_image_html(path):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''

    return '<img src="'+ path + '" style=max-width:100px " />'


def stimId_to_s3URL(domain, subdomain, stimID):
    
    if domain == 'structures':
        url =  "https://lax-{}-{}-all.s3.amazonaws.com/".format(domain, 
                                                                subdomain)\
               + "lax-{}-{}-{}-all.png".format(domain,
                                      subdomain,
                                      str(stimID).zfill(3))
    else: #check this
        url =  "https://lax-{}-{}-all.s3.amazonaws.com/".format(domain, 
                                                                subdomain)\
               + "lax-{}-{}-all-{}.png".format(domain,
                                      subdomain,
                                      str(stimID).zfill(3))

    return url
    

def stimId_to_html(stimId, domain = 'structures', subdomain = 'bridge'):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''
    stimURL = stimId_to_s3URL(domain, subdomain, stimId) 
    return '<img src="'+ stimURL + '" style=max-width:150px " />'




In [ ]:
# load dataframe

results_csv_directory = "../../results/csv/"
# df_trial = pd.read_csv(os.path.join(results_csv_directory, 'lax_corpus_1k_trial.csv'))
df_trial = pd.read_csv(os.path.join(results_csv_directory, 'lax_corpus_1k_trials_cogsci22.csv'))

## Preprocessing

### column name descriptions

```
id
'datatype': 
'iterationName':
'config_name':
    
'condition':
'domain': structures/ drawing
'subdomain': 
'gameID': uuid for participant

'shuffle':
'trialOrder':

'rt': reaction time
'rt_mins': reaction time in minutes

'trial_index': jspsych trial number (not experimental)
'trial_type':
'time_elapsed': 
'complete_dataset': did participant submit 10 responses?
'trial_num': trial number
    
'responses': complete response of what and where messages
'response_lists': same as above, but list of lists
'whats': list of what responses
'wheres': list of where responses
'n_steps': number of steps
'what_messages_lengths': list of lengths of what responses (characters)
'where_messages_lengths': list of lengths of where responses (characters)
'what_char_sum': total characters in what responses
'where_char_sum': total characters in where responses
'char_sum': total characters in responses 
'ppt_hit_8_step_limit': participant was in version of experiment with 8 steps, and hit this limit on at least one trial


'lemmatized_whats': lemmatized by spacy
'lemmatized_wheres':
'lemmatized_notstop_whats': lemmatized by spacy, stop words (incl numbers) removed
'lemmatized_notstop_wheres': 
'lemmatized_filtered_whats': lemmatized by spacy, determiners, punctuation and symbols removed
'lemmatized_filtered_wheres':

``` 

### Metadata    
```
'internal_node_id':
'view_history':
'stimId':
'stimURL':
'stim_group':
'partitionFamily':
'splitNumber':
'stimIDs':
'stimURLS':
'stimGroups':
'numGames':
'experimentType':
'experimentName':
'versionInd':
```

### Common preprocessing

Most preprocessing is dealt with in ./lax_corpus_data_generator.ipynb

Here we add preprocessing steps common to several but not all analyses

In [ ]:
# interpret more complex data structures i.e. lists
for column_name in ['responses',
                    'whats',
                    'wheres',
                    'lemmatized_whats',
                    'lemmatized_notstop_whats',
                    'lemmatized_filtered_whats',
                    'lemmatized_wheres',
                    'lemmatized_notstop_wheres',
                    'lemmatized_filtered_wheres',
                    'low_level_parts',
                    'mid_level_parts',
                    'high_level_parts',
                    'low_level_part_types',
                    'mid_level_part_types',
                    'high_level_part_types',
                    'low_level_part_params',
                    'mid_level_part_params',
                    'high_level_part_params',
                    'dreamcoder_program_dsl_0_tokens'
                   ]:
    df_trial[column_name] = df_trial[column_name].apply(ast.literal_eval)


In [ ]:
assert df_trial.dreamcoder_program_dsl_0_tokens.apply(lambda x: type(x) == list).all()

df_trial.loc[:,'base_program_length'] = df_trial.dreamcoder_program_dsl_0_tokens.apply(len)

In [ ]:
# add mean word count for each stim
what_word_sum_means = df_trial.groupby(['domain','subdomain','stimId']).mean()['what_word_sum'].reset_index()
what_word_sum_means = what_word_sum_means.rename(columns={'what_word_sum':'what_word_mean'})

#add means to df_trial (only do this if you will take one row per item from df_trial)
df_trial = df_trial.merge(what_word_sum_means, how='left', on=['domain','subdomain','stimId']) 

In [ ]:
from collections import defaultdict

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
word_to_pos = {}
for _, row in df_trial.iterrows():
    whats_list = row["lemmatized_whats"]
    pos_list = ast.literal_eval(row["whats_pos"])
    for i in range(len(whats_list)):
        item = whats_list[i]
        for j in range(len(item)):
            word = whats_list[i][j]
            pos = pos_list[i][j]
            word_to_pos[word] = pos

In [ ]:
# Replace common misspellings (applied to top-word analyses below)
spelling_map = {w: w for w in word_to_pos.keys()}
spelling_map["boarder"] = "border"
spelling_map["centre"] = "center"
spelling_map["cirlce"] = "circle"
spelling_map["cirlcle"] = "circle"
spelling_map["colour"] = "color"
spelling_map["collumn"] = "column"
spelling_map["columb"] = "column"
spelling_map["colum"] = "column"
spelling_map["hexgon"] = "hexagon"
spelling_map["heaxgon"] = "hexagon"
spelling_map["heaxagon"] = "hexagon"
spelling_map["hexagin"] = "hexagon"
spelling_map["hexogram"] = "hexagon"
spelling_map["horiz"] = "horizontal"
spelling_map["octogon"] = "octagon"
spelling_map["octogan"] = "octagon"
spelling_map["rec"] = "rectangle"
spelling_map["rect"] = "rectangle"
spelling_map["rectagle"] = "rectangle"
spelling_map["recagle"] = "rectangle"
spelling_map["sqaure"] = "square"
spelling_map["squae"] = "square"
spelling_map["squar"] = "square"
spelling_map["sqar"] = "square"
spelling_map["sqare"] = "square"
spelling_map["squre"] = "square"
spelling_map["verticle"] = "vertical"

In [ ]:
df_trial.to_csv(os.path.join(results_csv_directory, 'lax_corpus_1k_trials_cogsci22_preprocessed.csv'))

## Analysis of programs

- gallery: longest / average-length / shortest programs in each domain
- length: 
- diversity domain-specificity
- program token-level distinctiveness
- in: domain, subdomain, library_0 vs. library_compressive


In [ ]:
# use separate df with one entry per item
df_programs =  pd.read_csv('../../results/csv/lax_corpus_1k_programs_cogsci22.csv')

In [ ]:
df_programs['base_program_length'] = df_programs.dreamcoder_program_dsl_0_tokens.apply\
            (lambda x: len(ast.literal_eval(x)))

In [ ]:
# find order of complexity for subdomains
df_programs.groupby(['domain','subdomain'])['base_program_length'].apply(np.mean)

In [ ]:
# display items for each structures subdomain with shortest program

top_n = 10

for domain in ['structures']:
    for subdomain in subdomains[domain]:
        
        df_subdomain = df_programs[(df_programs.domain == domain) & (df_programs.subdomain == subdomain)]\
                        [['domain','subdomain','stimId','base_program_length']].sort_values('base_program_length',ascending=True)
        
#         grouped_df_list = [group_by_stim_id(df, config_name) for (config_name, df) in {subdomain: df_subdomain}.items()]
#         reduced_df = reduce(lambda x, y: pd.merge(x, y, on = ['stimId','domain','subdomain']), grouped_df_list).drop_duplicates()

        
        display(HTML(df_subdomain.head(top_n)\
                        .to_html(escape=False,
                                formatters=dict(stimId=
                                                lambda x:(stimId_to_html(x, domain = domain, subdomain = subdomain))))
                        .replace("\\n","<br>=======<br><br>")))
        
        
        

In [ ]:
# display items for each structures subdomain with longest program

top_n = 10

for domain in ['structures']:
    for subdomain in subdomains[domain]:
        
        df_subdomain = df_programs[(df_programs.domain == domain) & (df_programs.subdomain == subdomain)]\
                        [['domain','subdomain','stimId','base_program_length']].sort_values('base_program_length',ascending=False)
        
#         grouped_df_list = [group_by_stim_id(df, config_name) for (config_name, df) in {subdomain: df_subdomain}.items()]
#         reduced_df = reduce(lambda x, y: pd.merge(x, y, on = ['stimId','domain','subdomain']), grouped_df_list).drop_duplicates()

        
        display(HTML(df_subdomain.head(top_n)\
                        .to_html(escape=False,
                                formatters=dict(stimId=
                                                lambda x:(stimId_to_html(x, domain = domain, subdomain = subdomain))))
                        .replace("\\n","<br>=======<br><br>")))
        
        
        

In [ ]:
# display items for each structures subdomain with shortest program

top_n = 10

for domain in ['drawing']:
    for subdomain in subdomains[domain]:
        
        df_subdomain = df_programs[(df_programs.domain == domain) & (df_programs.subdomain == subdomain)]\
                        [['domain','subdomain','stimId','base_program_length']].sort_values('base_program_length',ascending=True)
        
        display(HTML(df_subdomain.head(top_n)\
                        .to_html(escape=False,
                                formatters=dict(stimId=
                                                lambda x:(stimId_to_html(x, domain = domain, subdomain = subdomain))))
                        .replace("\\n","<br>=======<br><br>")))
        
        
        

In [ ]:
# display items for each structures subdomain with longest program

top_n = 10

for domain in ['drawing']:
    for subdomain in subdomains[domain]:
        
        df_subdomain = df_programs[(df_programs.domain == domain) & (df_programs.subdomain == subdomain)]\
                        [['domain','subdomain','stimId','base_program_length']].sort_values('base_program_length',ascending=False)
        
#         grouped_df_list = [group_by_stim_id(df, config_name) for (config_name, df) in {subdomain: df_subdomain}.items()]
#         reduced_df = reduce(lambda x, y: pd.merge(x, y, on = ['stimId','domain','subdomain']), grouped_df_list).drop_duplicates()

        
        display(HTML(df_subdomain.head(top_n)\
                        .to_html(escape=False,
                                formatters=dict(stimId=
                                                lambda x:(stimId_to_html(x, domain = domain, subdomain = subdomain))))
                        .replace("\\n","<br>=======<br><br>")))
        
        
        

## Analysis of language
 
- Visualize: longest / average-length / shortest word counts in each domain (see: lax-corpus-results-visualizer.ipynb)
- token-level diversity: across domains, across subdomains within domain, across stims within subdomain, across participants
- token-level distinctiveness (PMI, tf-idf): across domains, across subdomains within domain, across stims within subdomain, across participants
- same as above, but now on "semantic" representations: gLoVe embeddings / BERT / & co. from huggingface / Spacy has all of these i think-> show a tsne


### Characterizing language use

#### Number of steps in instructions

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=df_trial, 
             x="n_steps", 
             hue="domain",
             hue_order=['drawing','structures'],
             binwidth=1,
             stat='proportion')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.title('number of steps in instructions')
# plt.savefig('./plots/instruction_steps_dist.pdf')

In [ ]:
d_steps = df_trial[df_trial.domain=='drawing']['n_steps']
s_steps = df_trial[df_trial.domain=='structures']['n_steps']

stats.ttest_ind(d_steps,s_steps)

In [ ]:
## over time
plt.figure(figsize=(10,6))
sns.lineplot(data=df_trial[(df_trial.complete_dataset)], 
             x='trial_num',
             y='n_steps', 
             hue='domain')
plt.ylim((1,11))
plt.title('total characters across trials, by domain')

#### Character count

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=df_trial, 
             x="what_char_sum", 
             hue="domain",
             hue_order=['drawing','structures'],
             binwidth=20,
             stat='proportion')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.title('total characters in instructions')
# plt.savefig('./plots/instruction_chars_dist.pdf')

In [ ]:
d_chars = df_trial[df_trial.domain=='drawing']['char_sum']
s_chars = df_trial[df_trial.domain=='structures']['char_sum']

stats.ttest_ind(d_chars,s_chars)

In [ ]:
## over time
plt.figure(figsize=(10,6))
sns.lineplot(data=df_trial[(df_trial.complete_dataset)], 
             x='trial_num', 
             y='char_sum', 
             hue='domain')
# plt.ylim((0,275))
plt.title('total characters across trials, by domain')


In [ ]:
# over time
plt.figure(figsize=(10,6))
sns.lineplot(data=df_trial[(df_trial.stimId != 'demo_stim') & 
                           (df_trial.complete_dataset)], 
             x='trial_num', 
             y='char_sum', 
             hue='subdomain')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.title('total characters across trials, by subdomain')

In [ ]:
## over time
plt.figure(figsize=(10,6))
sns.lineplot(data=df_trial[(df_trial.complete_dataset)], 
             x='trial_num', 
             y='what_char_sum', 
             hue='domain')
# plt.ylim((0,275))
plt.title('total WHAT characters across trials, by domain')


In [ ]:
# over time
plt.figure(figsize=(10,6))
sns.lineplot(data=df_trial[(df_trial.stimId != 'demo_stim') & 
                           (df_trial.complete_dataset)], 
             x='trial_num', 
             y='what_char_sum', 
             hue='subdomain')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.title('total WHAT characters across trials, by subdomain')

In [ ]:
plt.figure(figsize=(10,6))

sns.lineplot(data=df_trial[(df_trial.stimId != 'demo_stim') & 
                           (df_trial.complete_dataset)], 
             x='trial_num',
             y='what_char_sum', 
             hue='domain', 
             linestyle='--')

sns.lineplot(data=df_trial[(df_trial.stimId != 'demo_stim') & 
                           (df_trial.complete_dataset)], 
             x='trial_num', y='where_char_sum', 
             hue='domain', 
             linestyle='-', 
             legend=False)
plt.title('total characters across trials, WHAT vs. WHERE, by domain')

### Word-based measures

In [ ]:
## over time

plt.figure(figsize=(4,6))
sns.barplot(data=df_trial[(df_trial.complete_dataset)], 
             x='domain', 
             y='n_whats_filtered')
plt.ylabel('unique what words')
# plt.ylim((0,275))
plt.title('number of unique words used per response')
# plt.savefig('./plots/unique_whats_domain.pdf')

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=df_trial, 
             x="n_whats_filtered", 
             hue="domain",
             hue_order=['drawing','structures'],
             binwidth=1,
             stat='proportion')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.title('number of steps in instructions')
# plt.savefig('./plots/instruction_steps_dist.pdf')

In [ ]:
d_whats_filtered = df_trial[df_trial.domain=='drawing']['n_whats_filtered']
s_whats_filtered = df_trial[df_trial.domain=='structures']['n_whats_filtered']

stats.ttest_ind(d_whats_filtered,s_whats_filtered)

In [ ]:
s_whats_filtered.mean()

In [ ]:
d_whats_filtered.mean()

In [ ]:
## over time

plt.figure(figsize=(4,6))
sns.barplot(data=df_trial[(df_trial.complete_dataset)], 
             x='domain', 
             y='n_unique_whats')
plt.ylabel('unique what words')
# plt.ylim((0,275))
plt.title('number of unique words used per response')
# plt.savefig('./plots/unique_whats_domain.pdf')

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=df_trial, 
             x="n_unique_whats", 
             hue="domain",
             hue_order=['drawing','structures'],
             binwidth=1,
             stat='proportion')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.title('number of steps in instructions')
# plt.savefig('./plots/instruction_steps_dist.pdf')

In [ ]:
d_unique_whats = df_trial[df_trial.domain=='drawing']['n_unique_whats']
s_unique_whats = df_trial[df_trial.domain=='structures']['n_unique_whats']

stats.ttest_ind(d_unique_whats,s_unique_whats)

In [ ]:
## over time
plt.figure(figsize=(6,6))
sns.barplot(data=df_trial[(df_trial.complete_dataset)], 
             x='subdomain', 
             y='n_unique_whats',
             hue='domain')
plt.ylabel('unique what words')
plt.xticks(rotation = 45)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
# plt.ylim((0,275))
plt.title('number of unique words used per response')
# plt.savefig('./plots/unique_whats_subdomain.pdf')

In [ ]:
## over time

plt.figure(figsize=(10,6))
sns.lineplot(data=df_trial[(df_trial.complete_dataset)], 
             x='trial_num', 
             y='n_unique_whats', 
             hue='domain')
# plt.ylim((0,275))
plt.title('number of unique words used per response, over time')

#### Comparisons between subdomains

In [ ]:
plt.figure(figsize=(4,8))


sns.barplot(
            data = df_trial,
            x = 'subdomain',
            order = subdomains['drawing'] +  subdomains['structures'],
            palette= {**domain_palettes['drawing'],**domain_palettes_light['structures']},
            y = 'what_word_sum')
_ = plt.xticks(rotation = 60)
# plt.savefig('./plots/what_word_sum_subdomains.pdf')

In [ ]:
plt.figure(figsize=(8,6))

sns.violinplot(
    data = df_trial,
    x = 'subdomain',
    order = subdomains['drawing'] +  subdomains['structures'],
    y = 'what_word_sum',
    palette= {**domain_palettes['drawing'],**domain_palettes_light['structures']},
    linewidth=2)

In [ ]:
f = plt.figure(figsize=(8,6))

sns.violinplot(
    data = df_trial.groupby(['domain','subdomain','stimId']).first().reset_index(),
    x = 'subdomain',
    order = subdomains['drawing'] +  subdomains['structures'],
    palette = {**domain_palettes['drawing'],**domain_palettes_light['structures']},
    y = 'what_word_mean',
    linewidth=2)

### Word counts

token-based length: across domains, across subdomains within domain, across stims within subdomain, across participants

In [ ]:
sns.histplot(data = df_trial, x="what_word_sum", log_scale=True, fill=False, element="step")
sns.histplot(data = df_trial, x="where_word_sum", log_scale=True, fill=False, element="step")

In [ ]:
sns.histplot(data = df_trial,
             x="what_word_sum", 
             hue='domain',
             stat="density",
             common_norm=False,
             log_scale=True,
             fill=False,
             element="step")

In [ ]:
# mean word count (across participants) for each subdomain
plt.figure(figsize=(4,6))

sns.barplot(data=what_word_sum_means,
            x='domain',
            y='what_word_mean',
            hue_order=['structures','drawing']
           )
plt.xticks(rotation = 45)
plt.ylabel('what words')
plt.title('word count by domain')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.subplots_adjust(left=0.2, bottom=0.35)
# plt.savefig('./plots/what_word_count_domain.pdf')

In [ ]:
# mean word count (across participants) for each subdomain
plt.figure(figsize=(6,6))

sns.barplot(data=what_word_sum_means,
            x='subdomain',
            hue='domain',
            y='what_word_mean',
            hue_order=['drawing','structures']
           )
plt.xticks(rotation = 45)
plt.ylabel('what words')
plt.title('word count by subdomain')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.subplots_adjust(left=0.2, bottom=0.35)
# plt.savefig('./plots/what_word_count_subdomain.pdf')

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=df_trial[df_trial.subdomain.isin(['nuts-bolts','dials'])], 
             x="what_word_mean", 
             hue="subdomain",
#              hue_order=['drawing','structures'],
             binwidth=5,
             stat='proportion')
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.title('mean number of words in instructions')
# plt.savefig('./plots/instruction_chars_dist.pdf')

## Top words in domain (by counts)

In [ ]:
# by domain
all_words = {}
top_words_domain = {}
n_words_in_domain = {}
tf = {}
df = {}

all_docs = []

for domain in domains:
    
    doc = [d for sublist in df_trial[(df_trial.domain==domain) &
                                     (df_trial.complete_dataset)]['lemmatized_filtered_whats'] 
                                              for item in sublist
                                              for d in item]
    
    doc = [word for word in doc if word_to_pos[word]!='NUM']
    c = Counter(doc)
    top_words_domain[domain] = c.most_common(10)
    

In [ ]:
df_top_words = pd.DataFrame(top_words_domain)

In [ ]:
X_OFFSET_START = 0
Y_OFFSET_START = 1

X_OFFSET_INTERVAL = 0.5
Y_OFFSET_INTERVAL = 0.1

X_OFFSET_WORD = 0.13

cmap = matplotlib.cm.get_cmap("Greys")

fig = plt.figure()

x_offset = X_OFFSET_START
for domain in domains:
    
    y_offset = Y_OFFSET_START

#     plt.text(x_offset, y_offset, domain, fontweight="bold", color=cmap(1.0))
    y_offset -= Y_OFFSET_INTERVAL
    
    domain_counts = np.array([b for (a, b) in top_words_domain[domain]])
    
    norm = matplotlib.colors.Normalize(vmin=min(-(domain_counts.mean()*3), domain_counts.min()), vmax=domain_counts.max())
    
    for word, count in top_words_domain[domain]:
        alpha = 1
        plt.text(x_offset, y_offset, f"({count:.0f}) ", color=cmap(norm(count)), fontsize=12, alpha=alpha, fontname="Arial")
        plt.text(x_offset + X_OFFSET_WORD, y_offset, word, color=cmap(norm(count)), fontsize=16, alpha=alpha)
        
        y_offset -= Y_OFFSET_INTERVAL
    
    x_offset += X_OFFSET_INTERVAL
    
plt.grid(False)
plt.axis("off")
plt.show()

fig.savefig(f"top_words.pdf", bbox_inches="tight")


## Most diagnostic words of subdomain (PMI)

In [ ]:
DOMAIN = "structures"
# DOMAIN = "drawing"

df_domain = df_trial[(df_trial.domain == DOMAIN) & (df_trial.complete_dataset) & (~df_trial.ppt_hit_8_step_limit) & (df_trial.stimId != 'demo_stim')]
df_domain = df_domain.reset_index(drop=True)

df_domain["lemmatized_whats_flat"] = df_domain["lemmatized_whats"].map(lambda whats_list: " ".join([spelling_map[item] for sublist in whats_list for item in sublist if word_to_pos[item] == "NOUN"]))


df_domain

In [ ]:
vectorizer = CountVectorizer(strip_accents="unicode", min_df=5, stop_words="english")
X = vectorizer.fit_transform(df_domain["lemmatized_whats_flat"])

df_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

PSEUDOCOUNT = 1 / len(df_counts.columns)

df_counts = df_counts + PSEUDOCOUNT
df_counts

In [ ]:
d_pmi = defaultdict(dict)

N = df_counts.sum().sum()
JOINT_EXP = 1

subdomain_priors = ((df_domain.subdomain.value_counts()) / len(df_domain)).to_dict()

for subdomain in subdomain_priors:
    p_subdomain = np.log2(subdomain_priors[subdomain])
    for word in df_counts.columns:
        p_joint = np.log2((df_counts[word][df_domain.subdomain == subdomain].sum()) / N)
        p_word = np.log2((df_counts[word].sum()) / N)
                
        pmi = p_joint - (p_word + p_subdomain)
        d_pmi[subdomain][word] = pmi

In [ ]:
# Percentage of positive PMI values
df_pmi = pd.DataFrame(d_pmi)
(df_pmi > 0).sum().sum() / (df_pmi != 0).sum().sum()

In [ ]:
df_pmi

In [ ]:
pd.DataFrame({subdomain: df_pmi[subdomain].nlargest(30).index.tolist() for subdomain in subdomain_priors})

In [ ]:
# reorder columns
df_pmi = df_pmi[subdomains[DOMAIN]]

In [ ]:
df_pmi

In [ ]:
TOP_N = 10

X_OFFSET_START = 0
Y_OFFSET_START = 1

X_OFFSET_INTERVAL = 0.35
Y_OFFSET_INTERVAL = 0.1

X_OFFSET_WORD = 0.11

# CMAPS = map(matplotlib.cm.get_cmap, ["Blues", "Oranges", "Greens", "Reds"])
CMAPS = gradients


fig = plt.figure()

x_offset = X_OFFSET_START
for subdomain, cmap in zip(df_pmi.columns, CMAPS):
    df_pmi_subdomain_top = df_pmi[subdomain].nlargest(TOP_N)
    
    y_offset = Y_OFFSET_START

#     plt.text(x_offset, y_offset, subdomain, fontweight="bold", color=cmap(1.0), fontname="Arial")
    y_offset -= Y_OFFSET_INTERVAL
    
    norm = matplotlib.colors.Normalize(vmin=min(0, df_pmi_subdomain_top.min()), vmax=df_pmi_subdomain_top.max())
    
    for word, pmi in df_pmi_subdomain_top.iteritems():
        alpha = int(pmi > 0)
        plt.text(x_offset, y_offset, f"({pmi:.2f}) ", color=cmap(norm(pmi)), fontsize=12, alpha=alpha, fontname="Arial")
        plt.text(x_offset + X_OFFSET_WORD, y_offset, word, color=cmap(norm(pmi)), fontsize=16, alpha=alpha)
        
        y_offset -= Y_OFFSET_INTERVAL
    
    x_offset += X_OFFSET_INTERVAL
    
plt.grid(False)
plt.axis("off")
plt.show()

fig.savefig(f"pmi_{DOMAIN}.pdf", bbox_inches="tight")

## Analysis of programs x language

- Scatterplot: for each stim AND baseDSL vs. ‘compressive’ DSLL: word count vs. program length; |set(words)| vs.  |set(program_tokens)|
  - Questions re: overall trend (sublinear?)
  - Questions re: where we see high variation in language length given baseDSL program length 
  - We can further break these down in: domain, subdomain, annotator, library_0 vs. library_compressive


Let's merge the df_structures data frame with trial

In [ ]:
#fig = sns.scatterplot(x='n_blocks', y = 'what_char_sum', alpha=0.5, data=df_combined)
g = sns.jointplot(x='n_blocks', y = 'what_word_sum', alpha = 0.2,  data=df_trial)
#g.ax_joint.set_xscale('log')
#g.ax_joint.set_yscale('log')
#fig.plot([0, 100], [0, 100])

In [ ]:
df_trial.query('what_word_sum == 1')['responses']

### n stroke vs mean word count for gadget item

In [ ]:
# structures, n blocks vs mean word count for that stim

# just grab means (i.e. only one row per item needed)
df =  df_trial[df_trial.domain == 'drawing'].groupby(['domain','subdomain','stimId']).first().reset_index()

plt.figure(figsize=(10,10))

s = sns.scatterplot(data = df,
                x = 'n_strokes',
                y = 'what_word_mean',
                hue='subdomain',
                alpha=0.6)

# plt.title("number of words by gadget complexity")
plt.xlabel("strokes in image")
plt.ylabel("mean number of words")

s.plot([0,1],[0,1], 
       transform=s.transAxes, 
       color='grey',
       linestyle='--')

# plt.savefig('./plots/gadget_language_item_complexity.pdf')

In [ ]:
# structures, n blocks vs mean word count for that stim

# just grab means (i.e. only one row per item needed)
df = df_trial[df_trial.domain == 'drawing'].groupby(['domain','subdomain','stimId']).first().reset_index()

s = sns.FacetGrid(data = df,
                  col='subdomain', 
                  hue='subdomain',
                  height=5, 
                  aspect=0.85, # set aspect ratio here (although this includes titles, labels etc.)
                )

s.map(sns.scatterplot,
        'n_strokes',
        'what_word_mean',
        alpha=0.6)



for ax in s.axes_dict.values():
    ax.axline((0, 0), slope=1, c=".3", ls="--", zorder=0)
    ax.set(xlabel="strokes in image", ylabel="mean number of words")
    
# plt.savefig('./plots/gadget_language_item_complexity_facet.pdf')

### n blocks vs mean word count for structure item

In [ ]:
# structures, n blocks vs mean word count for that stim

# just grab means (i.e. only one row per item needed)
df =  df_trial[df_trial.domain == 'structures'].groupby(['domain','subdomain','stimId']).first().reset_index()

plt.figure(figsize=(10,10))

s = sns.scatterplot(data = df,
                x = 'n_blocks',
                y = 'what_word_mean',
                hue='subdomain',
                alpha=0.6)

plt.xlabel("blocks in structure")
plt.ylabel("mean number of words")

s.plot([0,1],[0,1], 
       transform=s.transAxes, 
       color='grey',
       linestyle='--')

# plt.savefig('./plots/structure_language_item_complexity.pdf')

In [ ]:
# structures, n blocks vs mean word count for that stim

# just grab means (i.e. only one row per item needed)
df = df_trial[df_trial.domain == 'structures'].groupby(['domain','subdomain','stimId']).first().reset_index()

s = sns.FacetGrid(data = df,
                  col='subdomain', 
                  hue='subdomain',
                  height=5, 
                  aspect=0.85, # set aspect ratio here (although this includes titles, labels etc.)
                )

s.map(sns.scatterplot,
        'n_blocks',
        'what_word_mean',
        alpha=0.6)

for ax in s.axes_dict.values():
    ax.axline((0, 0), slope=1, c=".3", ls="--", zorder=0)
    ax.set(xlabel="blocks in structure", ylabel="mean number of words")
    
# plt.savefig('./plots/structure_language_item_complexity_facet.pdf')

### base program length vs mean word count for gadget item

In [ ]:
# structures, n blocks vs mean word count for that stim

# just grab means (i.e. only one row per item needed)
df =  df_trial[df_trial.domain == 'drawing'].groupby(['domain','subdomain','stimId']).first().reset_index()

plt.figure(figsize=(10,10))

s = sns.scatterplot(data = df,
                x = 'base_program_length',
                y = 'what_word_mean',
                hue='subdomain',
                alpha=0.6)

plt.xlabel("base program length")
plt.ylabel("mean number of words")

# s.plot([0,1],[0,1], 
#        transform=s.transAxes, 
#        color='grey',
#        linestyle='--')

# plt.savefig('./plots/gadget_language_item_complexity.pdf')

In [ ]:
# structures, n blocks vs mean word count for that stim

# just grab means (i.e. only one row per item needed)
# remove one outlier to scale graphs
df = df_trial[(df_trial.domain == 'drawing') & (df_trial.what_word_mean < 125)].groupby(['domain','subdomain','stimId']).first().reset_index()

s = sns.FacetGrid(data = df,
                  col='subdomain', 
                  hue='subdomain',
                  height=5, 
                  aspect=0.8, # set aspect ratio here (although this includes titles, labels etc.)
                  sharex=False,
                  sharey=True,
                )

s.map(sns.scatterplot,
        'base_program_length',
        'what_word_mean',
        alpha=0.5)



for ax in s.axes_dict.values():
    ax.set_title(None)
    ax.set_ylabel(None)
    ax.set_xlabel(None)
#     ax.set_xlim([0,470])

# for ax in s.axes_dict.values():
#     ax.axline((0, 0), slope=1, c=".3", ls="--", zorder=0)
#     ax.set(xlabel="base_program_length", ylabel="mean number of words")
    
# plt.savefig('./plots/gadget_basedsl_langlength_facet.pdf')

In [ ]:
for subdomain in df["subdomain"].unique():
    print(subdomain)
    df[df["subdomain"] == subdomain]
    break

In [ ]:
df[df["subdomain"] == subdomain][["base_program_length", "what_word_mean"]].corr()

In [ ]:
scipy.stats.pearsonr(
    x=df[df["subdomain"] == subdomain]["base_program_length"],
    y=df[df["subdomain"] == subdomain]["what_word_mean"]
)

### n blocks vs mean word count for structure item

In [ ]:
# structures, n blocks vs mean word count for that stim

# just grab means (i.e. only one row per item needed)
df =  df_trial[df_trial.domain == 'structures'].groupby(['domain','subdomain','stimId']).first().reset_index()

plt.figure(figsize=(10,10))

s = sns.scatterplot(data = df,
                x = 'base_program_length',
                y = 'what_word_mean',
                hue='subdomain',
                alpha=0.6)

plt.xlabel("base_program_length")
plt.ylabel("mean number of words")

# s.plot([0,1],[0,1], 
#        transform=s.transAxes, 
#        color='grey',
#        linestyle='--')

# plt.savefig('./plots/structure_language_item_complexity.pdf')

In [ ]:
# structures, n blocks vs mean word count for that stim

# just grab means (i.e. only one row per item needed)
df = df_trial[(df_trial.domain == 'structures') & (df_trial.what_word_mean < 150)].groupby(['domain','subdomain','stimId']).first().reset_index()

s = sns.FacetGrid(data = df,
                  col='subdomain', 
                  hue='subdomain',
                  height=5, 
                  aspect=0.8, # set aspect ratio here (although this includes titles, labels etc.)
                  sharex=False,
                  sharey=True,
                )

s.map(sns.scatterplot,
        'base_program_length',
        'what_word_mean',
        alpha=0.5)

for ax in s.axes_dict.values():
    ax.set_title(None)
    ax.set_ylabel(None)
    ax.set_xlabel(None)
#     ax.set_xlim([0,470])

# for ax in s.axes_dict.values():
#     ax.axline((0, 0), slope=1, c=".3", ls="--", zorder=0)
#     ax.set(xlabel="base_program_length", ylabel="mean number of words")
    
# plt.savefig('./plots/structures_basedsl_langlength_facet.pdf')

## Likelihood ratio test for linear vs. log models

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import scipy

In [ ]:
# https://www.statology.org/likelihood-ratio-test-in-python/

likelihood_test_results = []

for domain in ["drawing", "structures"]:
    df = df_trial[df_trial.domain == domain].groupby(['domain','subdomain','stimId']).first().reset_index()
    for subdomain in df["subdomain"].unique():
        y = df[df["subdomain"] == subdomain]["what_word_mean"]

        # Reduced model
        x = df[df["subdomain"] == subdomain]["base_program_length"]
        x = sm.add_constant(x)
        reduced_model = sm.OLS(y, x).fit()

        # Full model
        df["log_base_program_length"] = np.log(df["base_program_length"])

        x = df[df["subdomain"] == subdomain][["base_program_length", "log_base_program_length"]]
        x = sm.add_constant(x)
        full_model = sm.OLS(y, x).fit()

        #calculate likelihood ratio Chi-Squared test statistic
        LR_statistic = -2*(reduced_model.llf - full_model.llf)

        #calculate p-value of test statistic using 2 degrees of freedom
        p_val = scipy.stats.chi2.sf(LR_statistic, 2)

        likelihood_test_results.append({
            "domain": domain,
            "subdomain": subdomain,
            "chi-squared": LR_statistic,
            "p_val": p_val,
        })
        
df_likelihood_test_results = pd.DataFrame(likelihood_test_results)
df_likelihood_test_results

### Compare distributions of words across domain/ subdomains (JSD)

#### Between domains

In [ ]:
USE_COUNTS = True

In [ ]:
trial_whats = df_trial.groupby(['gameID','trial_num'])['lemmatized_filtered_whats'].apply(lambda trial_responses: \
    ([x for xs in [word for sublist in trial_responses for word in sublist] for x in xs]))
trial_whats_counts = trial_whats.apply(lambda x: Counter(x))

df_trial_whats = df_trial[['gameID','trial_num','subdomain', 'domain','lemmatized_filtered_whats']].groupby(['gameID','trial_num']).first()
df_trial_whats.loc[:,'trial_whats'] = trial_whats
df_trial_whats.loc[:,'what_counts'] = trial_whats_counts

all_words = np.unique([x for xs in trial_whats for x in xs])

In [ ]:
for w in all_words:
    if USE_COUNTS:
        df_trial_whats[w] = df_trial_whats['what_counts'].apply(lambda row: int(row[w])) # word counts
    else:
        df_trial_whats[w] = df_trial_whats['trial_whats'].apply(lambda row: int(w in row)) # present/absent
        
df_trial_whats = df_trial_whats.reset_index()

In [ ]:
# Calculate true JSD

# word counts for domains
drawing_counts = df_trial_whats.loc[(df_trial_whats['domain'] == 'drawing')].iloc[:,10:].sum(axis=0)
structures_counts = df_trial_whats.loc[(df_trial_whats['domain'] == 'structures')].iloc[:,10:].sum(axis=0)

true_jsd = distance.jensenshannon(drawing_counts,structures_counts,2)
print(true_jsd)

In [ ]:
# Calculate null distribution of JSDs
# JSD for distributions of words in domains
# Shuffle domain tags. 1000 random assingments to 2 (preserve sizes)

RANDOM_SEED = 0
n_iters = 1000

jsds = []

# calculate true split of trials into domains
domain_assignments = df_trial.domain.copy()

np.random.seed(RANDOM_SEED)
# for each iteration
for n in range(0, n_iters):
    
    # assign trial random domain tag (following partition of domains in data)
    np.random.shuffle(domain_assignments)
    
    drawing_counts = df_trial_whats.iloc[:,10:][domain_assignments == 'drawing'].sum(axis = 0)
    structures_counts = df_trial_whats.iloc[:,10:][domain_assignments == 'structures'].sum(axis = 0)
    jsd = distance.jensenshannon(drawing_counts,structures_counts,2)
    
    # calculate JSD
    jsds.append(jsd)


In [ ]:
plt.hist(jsds)

In [ ]:
# Report p-value. (how many are greater than true JSD)
(sum(jsds > true_jsd) / n_iters) * 2

#### Between subdomains

In [ ]:
domain = 'drawing'

In [ ]:
# Calculate true mean JSD
# One domain at a time
df_trial_whats_domain = df_trial_whats.loc[(df_trial_whats['domain'] == domain)].reset_index(drop=True).copy()

subdomain_counts = {}
subdomain_jsds = {}

# get counts
for subdomain in subdomains[domain]:
    subdomain_counts[subdomain] = df_trial_whats_domain.loc[(df_trial_whats['subdomain'] == subdomain)]\
                                    .iloc[:,10:].sum(axis=0)

# get JSDS
for subdomain_i in subdomains[domain]:
    subdomain_jsds[subdomain_i] = {}
    for subdomain_j in subdomains[domain]:
        subdomain_jsds[subdomain_i][subdomain_j] = distance.jensenshannon(subdomain_counts[subdomain_i],
                                                                          subdomain_counts[subdomain_j], 2)
        

true_subdomain_jsds = pd.DataFrame.from_dict(subdomain_jsds)
true_subdomain_jsds

In [ ]:
# get true mean JSD
true_mean_jsd = np.array(true_subdomain_jsds)[np.triu_indices(4,k = 1)].mean()
true_mean_jsd

In [ ]:
# Calculate null distribution of mean JSDs

df_trial_whats_domain = df_trial_whats.loc[(df_trial_whats['domain'] == domain)].reset_index(drop=True).copy()

mean_jsds = []

n_iters = 1000

# calculate true split of trials into domains
subdomain_assignments = df_trial_whats_domain.subdomain.copy()

np.random.seed(RANDOM_SEED)

for n in range(0,n_iters):
    
    np.random.shuffle(subdomain_assignments)

    subdomain_counts = {}
    subdomain_jsds = {}

    # get counts
    for subdomain in subdomains[domain]:
        subdomain_counts[subdomain] = df_trial_whats_domain.iloc[:,10:][subdomain_assignments == subdomain]\
                                        .sum(axis=0)

    # get JSDS
    for subdomain_i in subdomains[domain]:
        subdomain_jsds[subdomain_i] = {}
        for subdomain_j in subdomains[domain]:
            subdomain_jsds[subdomain_i][subdomain_j] = distance.jensenshannon(subdomain_counts[subdomain_i],
                                                                              subdomain_counts[subdomain_j], 2)


    subdomain_jsds = pd.DataFrame.from_dict(subdomain_jsds)
    mean_jsd = np.array(subdomain_jsds)[np.triu_indices(4,k = 1)].mean()
    mean_jsds.append(mean_jsd)

In [ ]:
plt.hist(mean_jsds)

In [ ]:
# Report p-value. (how many are greater than true JSD)
(sum(mean_jsds > true_mean_jsd) / n_iters) * 2

## utils

### merge urls with top down abstraction dataframe

In [ ]:
urls = df_trial[df_trial.domain=='structures'].groupby(['stimId','stimURL','subdomain']).first().reset_index()[['blocks','stimId','stimURL','domain','subdomain']]
urls

In [ ]:
df_topdownabs = pd.read_csv('../../stimuli/towers/df_structures_topdownabs_consistent_abstractions.csv')

In [ ]:
df_topdownabs['stimId'] = df_topdownabs['structure_number']
df_topdownabs['subdomain'] = df_topdownabs['structure_type']

In [ ]:
df_topdownabs.merge(urls, how='left',on=['blocks','stimId','subdomain'])

In [ ]:
df_topdownabs.to_csv('../../stimuli/towers/df_structures_topdownabs_consistent_abstractions.csv')